## Linear Regression using pytorch built-ins

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('linear.csv')
df.head()

,Temperature,Humidity,WindSpeed,Visibility,Pressure
0,9.472222,0.89,14.1197,15.8263,1015.13
1,9.355556,0.86,14.2646,15.8263,1015.63
2,9.377778,0.89,3.9284,14.9569,1015.94
3,8.288889,0.83,14.1036,15.8263,1016.41
4,8.755556,0.83,11.0446,15.8263,1016.51


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
# Predicting the temperature based on other features
inputs = df.drop('Temperature',axis=1).values
inputs = inputs.astype('float32')
target = df.Temperature.values
target = target.astype('float32')

In [5]:
X_train, X_test, y_train, y_test = train_test_split( inputs, target, test_size=0.1, random_state=42)

###### Convert numpy to tensor

In [6]:
inputs = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)
test_inputs = torch.from_numpy(X_test)
test_target = torch.from_numpy(y_test)

In [7]:
from torch.utils.data import TensorDataset

In [8]:
train = TensorDataset(inputs,target)

###### Data Loader helps to split up into batches 
Because we are going give the model a huge amount of data

In [9]:
from torch.utils.data import DataLoader
batch_size = 32
dl = DataLoader(train, batch_size, shuffle=True)

In [10]:
for x,y in dl:
    print('batch :' )
    print(x)
    print(y)
    break

batch :
tensor([[8.6000e-01, 7.1484e+00, 1.1061e+01, 1.0040e+03],
        [3.4000e-01, 3.2522e+00, 1.6100e+01, 1.0154e+03],
        [5.2000e-01, 1.7823e+01, 1.5826e+01, 1.0158e+03],
        [8.9000e-01, 6.1502e+00, 9.7566e+00, 1.0155e+03],
        [7.1000e-01, 1.6986e+01, 1.5826e+01, 1.0132e+03],
        [7.2000e-01, 2.0528e+01, 1.0046e+01, 1.0097e+03],
        [8.7000e-01, 4.8461e+00, 5.1520e-01, 1.0315e+03],
        [9.6000e-01, 6.4400e+00, 9.7405e+00, 1.0205e+03],
        [7.8000e-01, 6.2790e+00, 1.5826e+01, 1.0160e+03],
        [9.0000e-01, 4.8300e+00, 1.5134e+01, 1.0218e+03],
        [5.4000e-01, 1.3508e+01, 1.5826e+01, 1.0216e+03],
        [7.6000e-01, 6.3756e+00, 1.6100e+01, 1.0119e+03],
        [5.9000e-01, 1.9272e+01, 1.1399e+01, 1.0130e+03],
        [8.1000e-01, 1.4876e+01, 7.9695e+00, 1.0268e+03],
        [5.3000e-01, 6.9069e+00, 1.1399e+01, 1.0083e+03],
        [8.5000e-01, 0.0000e+00, 9.9820e+00, 0.0000e+00],
        [6.6000e-01, 1.2107e+01, 1.0046e+01, 1.0184e+03],
      

###### Linear model and initializing the weights and biases

In [11]:
model = nn.Linear(4, 1) # 4 input  and 1 output
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1739, -0.1386,  0.3378,  0.3572]], requires_grad=True)
Parameter containing:
tensor([0.4643], requires_grad=True)


In [12]:
preds = model(test_inputs)
preds

tensor([[365.9384],
        [365.2283],
        [367.0817],
        ...,
        [366.9157],
        [366.7019],
        [364.2708]], grad_fn=<AddmmBackward>)

###### Loss function

In [13]:
import torch.nn.functional as functional
mse = functional.mse_loss

In [14]:
mse(model(test_inputs),test_target)

tensor(123750.9375, grad_fn=<MseLossBackward>)

###### Optimizer

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#### TRAIN THE MODEL

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [16]:
def Train(data_loader,model,loss_func,optimizer,epochs):    
    for epoch in range(epochs):        
        for x,y in data_loader:            
            # 1. Generate predictions
            pred = model(x)            
            # 2. Calculate loss
            loss = loss_func(pred, y)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        print(f'Epoch [{epoch+1}/{epochs}]  ------ Loss : {loss}')

In [17]:
Train(dl,model,mse,optimizer,30)

Epoch [1/30]  ------ Loss : 119.55933380126953
Epoch [2/30]  ------ Loss : 67.31321716308594
Epoch [3/30]  ------ Loss : 82.00146484375
Epoch [4/30]  ------ Loss : 123.18421173095703
Epoch [5/30]  ------ Loss : 48.12145233154297
Epoch [6/30]  ------ Loss : 130.91159057617188
Epoch [7/30]  ------ Loss : 58.525943756103516
Epoch [8/30]  ------ Loss : 125.42140197753906
Epoch [9/30]  ------ Loss : 87.766845703125
Epoch [10/30]  ------ Loss : 78.9830551147461
Epoch [11/30]  ------ Loss : 50.44871139526367
Epoch [12/30]  ------ Loss : 56.8448600769043
Epoch [13/30]  ------ Loss : 140.05050659179688
Epoch [14/30]  ------ Loss : 113.74604034423828
Epoch [15/30]  ------ Loss : 106.28412628173828
Epoch [16/30]  ------ Loss : 82.95130920410156
Epoch [17/30]  ------ Loss : 55.56040954589844
Epoch [18/30]  ------ Loss : 108.8739242553711
Epoch [19/30]  ------ Loss : 79.78697967529297
Epoch [20/30]  ------ Loss : 93.8116226196289
Epoch [21/30]  ------ Loss : 120.15497589111328
Epoch [22/30]  ------

In [18]:
prediction = model(test_inputs)

In [19]:
test_target

tensor([28.1222, 10.0000, 14.4000,  ..., 28.0333, 10.5278,  5.8056])

In [20]:
prediction

tensor([[15.0545],
        [14.6495],
        [14.7801],
        ...,
        [15.1452],
        [14.8978],
        [14.8827]], grad_fn=<AddmmBackward>)

In [21]:
error = mse(prediction,test_target)
print(error)

tensor(100.8846, grad_fn=<MseLossBackward>)
